In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import sys
import torch
from torch_DNN_class_v2 import  ANNModel


all_columns = ["Fy_steel", "E_steel", "Gama_steel", "Nu_steel", "tT_Top_tower", "tT_Bot_tower", 
               "thickness_monopile", "M_nacelle", "M_hub", "M_blade", "GammaSoil", "Phi_soil", 
               "G_soil", "VAve_Wind", "Hs_Wave", "Damping", 
               "TowerTopDisp_IM1_a", "TowerTopDisp_IM1_b", "TowerTopDisp_IM1_Beta", 
               "TowerTopAcc_IM2_a", "TowerTopDisp_IM2_b", "TowerTopDisp_IM2_Beta", 
               "TowerTopDisp_IM3_a", "TowerTopDisp_IM3_b", "TowerTopDisp_IM3_Beta"]

# Separate features and labels
features = all_columns[:16]
labels = all_columns[16:]

sheets = pd.read_excel(file_path, sheet_name=None)


dfs = []

for sheet_name, df in sheets.items():
    assert set(all_columns).issubset(df.columns), f"Missing columns in sheet {sheet_name}"

    df = df[all_columns]
    label_rename = {label: f"{label}_{sheet_name}" for label in labels}
    df = df.rename(columns=label_rename)

    dfs.append(df)

combined_df = pd.concat(dfs, axis=1)

final_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

expected_columns = len(features) + len(labels) * len(sheets)
assert final_df.shape[1] == expected_columns, f"Expected {expected_columns} columns, got {final_df.shape[1]}"

X = final_df[features].values
y = final_df.iloc[:, 16:].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

for n_components in range(1, len(features) + 1):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)  # Apply PCA to scaled features
    print(f"Number of components retained: {n_components}")
    print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
    print(f"Total explained variance: {sum(pca.explained_variance_ratio_)}")
    print()

optimal_n_components = 0.99 

pca = PCA(n_components=optimal_n_components)
X_pca = pca.fit_transform(X_scaled)

scaler = MinMaxScaler()
X_pca_scaled = scaler.fit_transform(X_pca)

X_train, X_test, y_train, y_test = train_test_split(X_pca_scaled, y, test_size=0.25, random_state=99, shuffle=True)
    
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test labels shape: {y_test.shape}")

Number of components retained: 1
Explained variance ratio: [0.1512764]
Total explained variance: 0.15127640050410007

Number of components retained: 2
Explained variance ratio: [0.1512764  0.11456862]
Total explained variance: 0.26584502204292565

Number of components retained: 3
Explained variance ratio: [0.1512764  0.11456862 0.11279784]
Total explained variance: 0.3786428581986182

Number of components retained: 4
Explained variance ratio: [0.1512764  0.11456862 0.11279784 0.07583122]
Total explained variance: 0.45447408233761805

Number of components retained: 5
Explained variance ratio: [0.1512764  0.11456862 0.11279784 0.07583122 0.06793412]
Total explained variance: 0.5224082050209912

Number of components retained: 6
Explained variance ratio: [0.1512764  0.11456862 0.11279784 0.07583122 0.06793412 0.06538964]
Total explained variance: 0.5877978457756571

Number of components retained: 7
Explained variance ratio: [0.1512764  0.11456862 0.11279784 0.07583122 0.06793412 0.06538964

In [2]:
import optuna
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import os
import csv
from optuna.samplers import TPESampler
from optuna.visualization import plot_param_importances, plot_optimization_history
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

np.random.seed(42)
# Create directories
model_save_dir = "optuna_DNN"
optuna_studies_dir = "optuna_studies"
loss_hist_dir = "loss_hist"
os.makedirs(model_save_dir, exist_ok=True)
os.makedirs(optuna_studies_dir, exist_ok=True)
os.makedirs(loss_hist_dir, exist_ok=True)



device='cuda'
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32).to(device), 
                            torch.tensor(y_train, dtype=torch.float32).to(device))
val_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32).to(device), 
                            torch.tensor(y_test, dtype=torch.float32).to(device))
        
input_size = X_train.shape[1]
output_size = y_train.shape[1]

criterion = nn.L1Loss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def DNN_configure(trial):
    num_layers = trial.suggest_int('num_hidden_layers', 4, 10)
    hidden_layers = [trial.suggest_categorical(f'neurons_layer_{i}', [128, 256, 512]) for i in range(num_layers)]
    
    dropout = {}
    for i in range(1, num_layers):
        use_dropout = trial.suggest_categorical(f'use_dropout_{i}', [True, False])
        if use_dropout:
            dropout[i] = trial.suggest_float(f'dropout_prob_{i}', 0.05, 0.5)
        else:
            dropout[i] = 0.0
    
    res = {}
    num_res_connections = trial.suggest_int('num_res_connections', 0, num_layers - 1)
    for i in range(num_res_connections):
        layer_index = trial.suggest_int(f'res_layer_index_{i}', 1, num_layers - 1)
        num_sources = trial.suggest_int(f'num_sources_{i}', 0, layer_index)
        sources = [trial.suggest_int(f'source_{i}_{j}', -1, layer_index - 1) for j in range(num_sources)]
        res[layer_index] = {'sources': sorted(list(set(sources))), 'type': 'linear'}
    res = dict(sorted(res.items()))
    
    hidden_activation = trial.suggest_categorical('hidden_activation', ['ReLU', 'ELU', 'GELU', 'SiLU', 'LeakyReLU'])
    
    return hidden_layers, dropout, res, hidden_activation

def objective(trial):
    hidden_layers, dropout, res, hidden_activation = DNN_configure(trial)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical('optimizer_name', ['AdamW', 'Adam'])
    weight_decay = trial.suggest_float("weight_decay", 1e-7, 1e-3, log=True) if trial.suggest_categorical("use_weight_decay", [True, False]) else 0.0
    
    model = ANNModel().create_model(
        input_size, output_size, hidden_layers,
        residual_connections=res,
        dropout_params=dropout,
        hidden_activation=hidden_activation,
        seed=trial.suggest_int("model_seed", 1, 9999),
        scale=False
    )

    model.to(device)
    
    batch_size=trial.suggest_categorical('batch_size', [4, 8, 16, 32, 64]) 

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=X_test.shape[0], shuffle=False)

    model, train_losses, valid_losses = ANNModel().train_model(
        train_loader,val_loader, 
        epochs=500, 
        learning_rate=learning_rate, 
        optimizer_name=optimizer_name,
        optimizer_params={}, 
        device=device, 
        criterion=criterion, 
        patience=50, 
        model=model, 
        l2_lambda=weight_decay
    )
    # Save the model
    model_save_path = os.path.join(model_save_dir, f"model_trial_{trial.number}.pth")
    torch.save(model, model_save_path)
    
    # Save training and validation losses in CSV format
    loss_save_path = os.path.join(loss_hist_dir, f"losses_trial_{trial.number}.csv")
    with open(loss_save_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Train Losses', 'Validation Losses'])
        for train_loss, valid_loss in zip(train_losses, valid_losses):
            writer.writerow([train_loss, valid_loss])
        
    return min(valid_losses)



# Callback function to log information during optimization
def log_best_trials(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value:.4f}")
    print(f"Best trial so far: {study.best_trial.number} with value: {study.best_trial.value:.4f}")

# Create the study object with persistence
storage_name = f"sqlite:///{os.path.join(optuna_studies_dir, 'dnn_optimization.db')}"
study_name = "dnn_optimization_study_v1"

sampler = TPESampler(n_startup_trials=40, seed=42)


study = optuna.create_study(
    direction='minimize',
    sampler=sampler,
    storage=storage_name,
    study_name=study_name,
    load_if_exists=True
)

# Run the optimization with callback
study.optimize(objective, n_trials=100, callbacks=[log_best_trials])

print("Best trial:")
print(study.best_trial.params)



[I 2024-08-24 02:35:12,672] A new study created in RDB with name: dnn_optimization_study_v1
[I 2024-08-24 02:35:18,367] Trial 0 finished with value: 0.05781478062272072 and parameters: {'num_hidden_layers': 6, 'neurons_layer_0': 128, 'neurons_layer_1': 128, 'neurons_layer_2': 128, 'neurons_layer_3': 256, 'neurons_layer_4': 128, 'neurons_layer_5': 256, 'use_dropout_1': False, 'use_dropout_2': False, 'use_dropout_3': False, 'use_dropout_4': True, 'dropout_prob_4': 0.28140549728612524, 'use_dropout_5': True, 'dropout_prob_5': 0.32339518335564726, 'num_res_connections': 1, 'res_layer_index_0': 1, 'num_sources_0': 1, 'source_0_0': 0, 'hidden_activation': 'ReLU', 'learning_rate': 0.00023233503515390126, 'optimizer_name': 'AdamW', 'use_weight_decay': True, 'weight_decay': 4.467752817973903e-05, 'model_seed': 3117, 'batch_size': 32}. Best is trial 0 with value: 0.05781478062272072.


Early stopping triggered after 270 epochs
Trial 0 finished with value: 0.0578
Best trial so far: 0 with value: 0.0578


[I 2024-08-24 02:35:26,703] Trial 1 finished with value: 0.05466629937291145 and parameters: {'num_hidden_layers': 10, 'neurons_layer_0': 512, 'neurons_layer_1': 256, 'neurons_layer_2': 256, 'neurons_layer_3': 128, 'neurons_layer_4': 512, 'neurons_layer_5': 256, 'neurons_layer_6': 512, 'neurons_layer_7': 512, 'neurons_layer_8': 256, 'neurons_layer_9': 128, 'use_dropout_1': False, 'use_dropout_2': False, 'use_dropout_3': False, 'use_dropout_4': True, 'dropout_prob_4': 0.37096015425034773, 'use_dropout_5': True, 'dropout_prob_5': 0.39693523097955247, 'use_dropout_6': False, 'use_dropout_7': True, 'dropout_prob_7': 0.09855114214698701, 'use_dropout_8': False, 'use_dropout_9': False, 'num_res_connections': 9, 'res_layer_index_0': 3, 'num_sources_0': 1, 'source_0_0': 2, 'res_layer_index_1': 3, 'num_sources_1': 0, 'res_layer_index_2': 3, 'num_sources_2': 0, 'res_layer_index_3': 9, 'num_sources_3': 8, 'source_3_0': 5, 'source_3_1': 7, 'source_3_2': 7, 'source_3_3': 0, 'source_3_4': 7, 'source

Early stopping triggered after 187 epochs
Trial 1 finished with value: 0.0547
Best trial so far: 1 with value: 0.0547


[I 2024-08-24 02:35:52,720] Trial 2 finished with value: 0.07237636297941208 and parameters: {'num_hidden_layers': 8, 'neurons_layer_0': 128, 'neurons_layer_1': 512, 'neurons_layer_2': 512, 'neurons_layer_3': 128, 'neurons_layer_4': 256, 'neurons_layer_5': 512, 'neurons_layer_6': 256, 'neurons_layer_7': 128, 'use_dropout_1': False, 'use_dropout_2': True, 'dropout_prob_2': 0.34699282071538057, 'use_dropout_3': True, 'dropout_prob_3': 0.28834276026020295, 'use_dropout_4': True, 'dropout_prob_4': 0.453747091078997, 'use_dropout_5': True, 'dropout_prob_5': 0.2025634059719153, 'use_dropout_6': False, 'use_dropout_7': True, 'dropout_prob_7': 0.40094399563593075, 'num_res_connections': 5, 'res_layer_index_0': 1, 'num_sources_0': 0, 'res_layer_index_1': 7, 'num_sources_1': 4, 'source_1_0': -1, 'source_1_1': -1, 'source_1_2': 4, 'source_1_3': -1, 'res_layer_index_2': 2, 'num_sources_2': 1, 'source_2_0': 1, 'res_layer_index_3': 5, 'num_sources_3': 1, 'source_3_0': 3, 'res_layer_index_4': 2, 'num

...

Early stopping triggered after 101 epochs
Trial 97 finished with value: 0.0889
Best trial so far: 41 with value: 0.0459
Early stopping triggered after 136 epochs


[I 2024-08-24 02:57:37,407] Trial 98 finished with value: 0.04867761954665184 and parameters: {'num_hidden_layers': 10, 'neurons_layer_0': 256, 'neurons_layer_1': 256, 'neurons_layer_2': 256, 'neurons_layer_3': 256, 'neurons_layer_4': 256, 'neurons_layer_5': 128, 'neurons_layer_6': 256, 'neurons_layer_7': 512, 'neurons_layer_8': 256, 'neurons_layer_9': 256, 'use_dropout_1': False, 'use_dropout_2': False, 'use_dropout_3': True, 'dropout_prob_3': 0.4175908499803172, 'use_dropout_4': True, 'dropout_prob_4': 0.05530067158562005, 'use_dropout_5': False, 'use_dropout_6': False, 'use_dropout_7': True, 'dropout_prob_7': 0.2785695935740006, 'use_dropout_8': False, 'use_dropout_9': True, 'dropout_prob_9': 0.1378753336369719, 'num_res_connections': 8, 'res_layer_index_0': 8, 'num_sources_0': 8, 'source_0_0': 5, 'source_0_1': 6, 'source_0_2': 6, 'source_0_3': 6, 'source_0_4': 0, 'source_0_5': 6, 'source_0_6': 1, 'source_0_7': 5, 'res_layer_index_1': 9, 'num_sources_1': 9, 'source_1_0': 7, 'source_

Trial 98 finished with value: 0.0487
Best trial so far: 41 with value: 0.0459


[I 2024-08-24 02:57:45,430] Trial 99 finished with value: 0.05237593501806259 and parameters: {'num_hidden_layers': 9, 'neurons_layer_0': 128, 'neurons_layer_1': 512, 'neurons_layer_2': 256, 'neurons_layer_3': 512, 'neurons_layer_4': 128, 'neurons_layer_5': 128, 'neurons_layer_6': 128, 'neurons_layer_7': 256, 'neurons_layer_8': 256, 'use_dropout_1': True, 'dropout_prob_1': 0.2403976612503179, 'use_dropout_2': False, 'use_dropout_3': False, 'use_dropout_4': True, 'dropout_prob_4': 0.13134592801477368, 'use_dropout_5': False, 'use_dropout_6': False, 'use_dropout_7': True, 'dropout_prob_7': 0.33426489112799257, 'use_dropout_8': False, 'num_res_connections': 7, 'res_layer_index_0': 7, 'num_sources_0': 6, 'source_0_0': 1, 'source_0_1': 5, 'source_0_2': 5, 'source_0_3': 5, 'source_0_4': 4, 'source_0_5': 5, 'res_layer_index_1': 7, 'num_sources_1': 6, 'source_1_0': 3, 'source_1_1': 5, 'source_1_2': 2, 'source_1_3': 5, 'source_1_4': 5, 'source_1_5': -1, 'res_layer_index_2': 7, 'num_sources_2': 

Early stopping triggered after 134 epochs
Trial 99 finished with value: 0.0524
Best trial so far: 41 with value: 0.0459
Best trial:
{'num_hidden_layers': 8, 'neurons_layer_0': 128, 'neurons_layer_1': 512, 'neurons_layer_2': 512, 'neurons_layer_3': 256, 'neurons_layer_4': 128, 'neurons_layer_5': 128, 'neurons_layer_6': 256, 'neurons_layer_7': 512, 'use_dropout_1': True, 'dropout_prob_1': 0.12547590010576132, 'use_dropout_2': False, 'use_dropout_3': True, 'dropout_prob_3': 0.4717949191034361, 'use_dropout_4': True, 'dropout_prob_4': 0.1370006814304564, 'use_dropout_5': False, 'use_dropout_6': False, 'use_dropout_7': False, 'num_res_connections': 6, 'res_layer_index_0': 6, 'num_sources_0': 0, 'res_layer_index_1': 3, 'num_sources_1': 1, 'source_1_0': 1, 'res_layer_index_2': 5, 'num_sources_2': 2, 'source_2_0': 3, 'source_2_1': 0, 'res_layer_index_3': 6, 'num_sources_3': 4, 'source_3_0': 1, 'source_3_1': -1, 'source_3_2': -1, 'source_3_3': 4, 'res_layer_index_4': 5, 'num_sources_4': 5, 'sou

In [5]:
import optuna
import optuna.visualization as vis
import os


def save_optimization_history(study):
    try:
        fig = vis.plot_optimization_history(study)
        fig.write_html(os.path.join(optuna_studies_dir, "optimization_history.html"))
    except Exception as e:
        print(f"Could not generate optimization history plot: {str(e)}")

def save_param_importances(study):
    try:
        fig = vis.plot_param_importances(study)
        fig.write_html(os.path.join(optuna_studies_dir, "param_importances.html"))
    except Exception as e:
        print(f"Could not generate parameter importances plot: {str(e)}")

def save_parallel_coordinate(study):
    try:
        fig = vis.plot_parallel_coordinate(study)
        fig.write_html(os.path.join(optuna_studies_dir, "parallel_coordinate.html"))
    except Exception as e:
        print(f"Could not generate parallel coordinate plot: {str(e)}")

def save_slice_plot(study):
    try:
        fig = vis.plot_slice(study)
        fig.write_html(os.path.join(optuna_studies_dir, "slice_plot.html"))
    except Exception as e:
        print(f"Could not generate slice plot: {str(e)}")

# After optimization
print("Best trial:")
print(study.best_trial.params)

# Generate and save visualizations
if len(study.trials) > 1:  # Only attempt to visualize if there's more than one trial
    save_optimization_history(study)
    save_param_importances(study)
    save_parallel_coordinate(study)
    save_slice_plot(study)
    print("Visualizations saved as HTML files in the optuna_studies directory")
else:
    print("Not enough trials to generate visualizations")

print(f"Study saved in {optuna_studies_dir}")

Best trial:
{'num_hidden_layers': 8, 'neurons_layer_0': 128, 'neurons_layer_1': 512, 'neurons_layer_2': 512, 'neurons_layer_3': 256, 'neurons_layer_4': 128, 'neurons_layer_5': 128, 'neurons_layer_6': 256, 'neurons_layer_7': 512, 'use_dropout_1': True, 'dropout_prob_1': 0.12547590010576132, 'use_dropout_2': False, 'use_dropout_3': True, 'dropout_prob_3': 0.4717949191034361, 'use_dropout_4': True, 'dropout_prob_4': 0.1370006814304564, 'use_dropout_5': False, 'use_dropout_6': False, 'use_dropout_7': False, 'num_res_connections': 6, 'res_layer_index_0': 6, 'num_sources_0': 0, 'res_layer_index_1': 3, 'num_sources_1': 1, 'source_1_0': 1, 'res_layer_index_2': 5, 'num_sources_2': 2, 'source_2_0': 3, 'source_2_1': 0, 'res_layer_index_3': 6, 'num_sources_3': 4, 'source_3_0': 1, 'source_3_1': -1, 'source_3_2': -1, 'source_3_3': 4, 'res_layer_index_4': 5, 'num_sources_4': 5, 'source_4_0': 3, 'source_4_1': 1, 'source_4_2': 2, 'source_4_3': 4, 'source_4_4': 3, 'res_layer_index_5': 5, 'num_sources_5'

[W 2024-08-24 03:03:30,659] Your study has only completed trials with missing parameters.


Visualizations saved as HTML files in the optuna_studies directory
Study saved in optuna_studies


In [4]:
# import os
# import shutil

# #                                        ____________  carefully run this !!! it remove your repositories! _____________
# def clear_directories(*dirs):
#     for dir_path in dirs:
#         if os.path.exists(dir_path):
#             # Remove all contents of the directory
#             for filename in os.listdir(dir_path):
#                 file_path = os.path.join(dir_path, filename)
#                 try:
#                     if os.path.isfile(file_path) or os.path.islink(file_path):
#                         os.unlink(file_path)
#                     elif os.path.isdir(file_path):
#                         shutil.rmtree(file_path)
#                 except Exception as e:
#                     print(f'Failed to delete {file_path}. Reason: {e}')
#             print(f"Cleared contents of {dir_path}")
#         else:
#             # If the directory doesn't exist, create it
#             os.makedirs(dir_path)
#             print(f"Created directory {dir_path}")

# # Usage
# model_save_dir = "optuna_DNN"
# optuna_studies_dir = "optuna_studies"
# loss_hist_dir="loss_hist"
# clear_directories(model_save_dir, optuna_studies_dir, loss_hist_dir)